In [2]:
import geopandas as gp
import pandas as pd
import os

# 2020 Presidential Democratic & Republican Vote Share Aggregated to 2022 Legislative Districts

## Background:
- Received a request wanting % Democratic vote share on the new legislative districts

## Approach:
- Combine all of the block-level VEST disaggregated 2020 general election results
- Filter down to the votes for just Biden and Trump
- Join to the national BAF
- Aggregate to new districts
- Create the vote share percentages

## Links to datasets used:
[2022 National Block Assignment File](https://redistrictingdatahub.org/dataset/national-block-assignment-file-for-2022-state-legislative-and-congressional-districts/)

Block-level 2020 election results (multiple links)

For a full 'raw-from-source' file, contact info@redistrictingdatahub.org

### Load and Clean Block-Level Votes and District Assignments

In [3]:
hold = gp.read_file("/Users/peterhorton/Documents/RDH/raw_data/disagg/ak_2020_gen_2020_blocks.zip")

In [4]:
# hold_list = []
# path = "/Users/peterhorton/Documents/RDH/raw_data/disagg/"
# for file in os.listdir(path):
#     if file != ".DS_Store":
#         print(file)
#         hold = gp.read_file(path+file)
#         hold = hold[["GEOID20","VAP_MOD","G20PRERTRU","G20PREDBID"]]
#         hold_list.append(hold)
        
# comb_data = pd.concat(hold_list)

# comb_data["GEOID20"] = comb_data["GEOID20"].astype(str).str.zfill(15)
# comb_data.to_csv("./combined_national_block_elections.csv", index = False)

In [5]:
comb_data = pd.read_csv("./combined_national_block_elections.csv")
comb_data["GEOID20"] = comb_data["GEOID20"].astype(str).str.zfill(15)

In [6]:
fips_state_abbrev_dict = {
 '01': 'AL',
 '02': 'AK',
 '04': 'AZ',
 '05': 'AR',
 '06': 'CA',
 '08': 'CO',
 '09': 'CT',
 '10': 'DE',
 '12': 'FL',
 '13': 'GA',
 '15': 'HI',
 '16': 'ID',
 '17': 'IL',
 '18': 'IN',
 '19': 'IA',
 '20': 'KS',
 '21': 'KY',
 '22': 'LA',
 '23': 'ME',
 '24': 'MD',
 '25': 'MA',
 '26': 'MI',
 '27': 'MN',
 '28': 'MS',
 '29': 'MO',
 '30': 'MT',
 '31': 'NE',
 '32': 'NV',
 '33': 'NH',
 '34': 'NJ',
 '35': 'NM',
 '36': 'NY',
 '37': 'NC',
 '38': 'ND',
 '39': 'OH',
 '40': 'OK',
 '41': 'OR',
 '42': 'PA',
 '44': 'RI',
 '45': 'SC',
 '46': 'SD',
 '47': 'TN',
 '48': 'TX',
 '49': 'UT',
 '50': 'VT',
 '51': 'VA',
 '53': 'WA',
 '54': 'WV',
 '55': 'WI',
 '56': 'WY',
 '60': 'AS',
 '66': 'GU',
 '69': 'MP',
 '72': 'PR',
 '78': 'VI'}

In [7]:
comb_data["STATEFP"] = comb_data["GEOID20"].astype(str).str[0:2]
comb_data["STATE"] = comb_data["STATEFP"].map(fips_state_abbrev_dict)

In [8]:
comb_data["COUNTYFP"] = comb_data["GEOID20"].str[:5]

In [9]:
countyfips = pd.read_csv("/Users/peterhorton/Documents/RDH/raw_data/census/PL_COUNTYFP_NAMES.csv", dtype = str)

In [10]:
fips_dict = dict(zip(countyfips["GEOID"], countyfips["NAME"]))

In [11]:
comb_data["CNTY_NAME"] = comb_data["COUNTYFP"].map(fips_dict).fillna("NOTHING")

In [12]:
comb_data[comb_data["CNTY_NAME"]=="NOTHING"]

,GEOID20,VAP_MOD,G20PRERTRU,G20PREDBID,STATEFP,STATE,COUNTYFP,CNTY_NAME


In [13]:
comb_data = comb_data[["GEOID20","STATEFP","STATE","COUNTYFP","CNTY_NAME","VAP_MOD","G20PRERTRU","G20PREDBID"]]

In [14]:
national_pop = pd.read_csv("/Users/peterhorton/Documents/RDH/raw_data/census/national_pl_2020_csv.csv")

In [17]:
national_pop["GEOID20"] = national_pop["GEOID20"].astype(str).str.zfill(15)

In [20]:
national_pop = national_pop[["GEOID20", "P0010001"]]

In [21]:
final_join = pd.merge(comb_data, national_pop, how = "outer", on = "GEOID20")

In [23]:
final_join = final_join[["GEOID20","STATEFP","STATE","COUNTYFP","CNTY_NAME","P0010001","VAP_MOD","G20PRERTRU","G20PREDBID"]]

In [26]:
final_join.rename(columns = {"P0010001":"TOT_POP"}, inplace = True)

In [31]:
final_join.to_csv("./national_20_blocks_20_D_R_pres_share/national_20_blocks_20_D_R_pres_share.csv", index = False)

In [33]:
fields_dict = {'GEOID20':"State Abbreviation",
               'STATEFP':"State FIP Code",
               'STATE':"State Abbreviation",
               'COUNTYFP':"County FIP Code",
               'CNTY_NAME':"County Name",
               'TOT_POP':"Total Pop. from 2020 Census",
               'VAP_MOD':"Voting Age Pop. minus Adult Incarcerated Pop. from 2020 Census",
               'G20PRERTRU':"Total 2020 Trump Votes",
               'G20PREDBID':"Total 2020 Biden Votes"}

In [34]:
### Create README

title = "2020 Presidential Democratic & Republican Vote Share on Nationwide 2020 Census Blocks"
retrieval_date = "03/20/23"
fields_dict = fields_dict
github_link = "https://github.com/nonpartisan-redistricting-datahub/Processing-Requests/"
file_folder = "./national_20_blocks_20_D_R_pres_share/"

def full_readme_text(title, retrieval_date, fields_dict, github_link):

#First section of README
    readme_p1 = '''{title}\n
## RDH Date Retrieval
{retrieval_date}

## Sources
ADD SOURCE DATA HERE

## Fields:
'''.format(title = title, retrieval_date = retrieval_date)

#Second section of README
    fields_table = pd.DataFrame.from_dict(fields_dict.items())
    fields_table.columns = ["Field Name", "Description"]
    readme_p2 = fields_table.to_string(formatters={'Description':'{{:<{}s}}'.format(fields_table['Description'].str.len().max()).format, 'Field Name':'{{:<{}s}}'.format(fields_table['Field Name'].str.len().max()).format}, index=False, justify = "left")

#Third section of README
    readme_p3 = '''\n
## Processing Steps
Visit the RDH GitHub and the processing script for this code [here]({github_link})

Please direct questions related to processing this dataset to info@redistrictingdatahub.org.
'''.format(github_link=github_link)
    
    full_readme = str(readme_p1)+str(readme_p2)+str(readme_p3)
    return full_readme

with open(file_folder+"README.txt", 'w') as tf:
        tf.write(full_readme_text(title, retrieval_date, fields_dict, github_link))



# Filling the below out later when we update 118 and 117

In [2]:
# Note: Created this file by combining our disaggregated 2020 VEST files
national_block_election = pd.read_csv("./raw-from-source/national_block_election.csv")
national_baf = pd.read_csv("./raw-from-source/national_baf_boundary/national_baf_boundary.csv")

national_baf["GEOID20"] = national_baf["GEOID20"].astype(str).str.zfill(15)
national_block_election["GEOID20"] = national_block_election["GEOID20"].astype(str).str.zfill(15)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_6835/4070597731.py:3: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  national_baf = pd.read_csv("./raw-from-source/national_baf_boundary/national_baf_boundary.csv")


### Join Files Together, Check Merge

In [3]:
combined = pd.merge(national_baf, national_block_election, how = "outer", on = "GEOID20", indicator = True)
combined["_merge"].value_counts()

both          8126956
left_only           0
right_only          0
Name: _merge, dtype: int64

### Aggregate to Districts, Filter Out "Unassigned Areas", Create Vote Shares

In [4]:
combined["CONG-ID"] = combined["STATE"] + "-" + combined["CONG"].astype(str)
combined_cong = combined.groupby(['STATE',"CONG","CONG-ID"])[['P0010001', 'VAP_MOD', 'G20PRERTRU', 'G20PREDBID']].sum()
combined_cong.reset_index(inplace = True, drop = False)

In [5]:
combined["SLDL-ID"] = combined["STATE"] + "-" + combined["SLDL"].astype(str)
combined_SLDL = combined.groupby(['STATE',"SLDL","SLDL-ID"])[['P0010001', 'VAP_MOD', 'G20PRERTRU', 'G20PREDBID']].sum()
combined_SLDL.reset_index(inplace = True, drop = False)

In [6]:
combined["SLDU-ID"] = combined["STATE"] + "-" + combined["SLDU"].astype(str)
combined_SLDU = combined.groupby(['STATE',"SLDU","SLDU-ID"])[['P0010001', 'VAP_MOD', 'G20PRERTRU', 'G20PREDBID']].sum()
combined_SLDU.reset_index(inplace = True, drop = False)

In [7]:
combined_SLDL = combined_SLDL[~(combined_SLDL["SLDL-ID"].str.contains("NO VALUE"))&~(combined_SLDL["SLDL-ID"].str.contains("NO SLDL"))]
combined_SLDU = combined_SLDU[~(combined_SLDU["SLDU-ID"].str.contains("NO VALUE"))&~(combined_SLDU["SLDU-ID"].str.contains("NO SLDU"))]
combined_cong = combined_cong[~combined_cong["CONG-ID"].str.contains("NO VALUE")]

In [8]:
combined_SLDL["REP_pct"] = round(combined_SLDL["G20PRERTRU"] / (combined_SLDL["G20PRERTRU"] + combined_SLDL["G20PREDBID"]),3)
combined_SLDL["DEM_pct"] = round(combined_SLDL["G20PREDBID"] / (combined_SLDL["G20PRERTRU"] + combined_SLDL["G20PREDBID"]),3) 

combined_SLDU["REP_pct"] = round(combined_SLDU["G20PRERTRU"] / (combined_SLDU["G20PRERTRU"] + combined_SLDU["G20PREDBID"]),3)
combined_SLDU["DEM_pct"] = round(combined_SLDU["G20PREDBID"] / (combined_SLDU["G20PRERTRU"] + combined_SLDU["G20PREDBID"]),3)

combined_cong["REP_pct"] = round(combined_cong["G20PRERTRU"] / (combined_cong["G20PRERTRU"] + combined_cong["G20PREDBID"]),3)
combined_cong["DEM_pct"] = round(combined_cong["G20PREDBID"] / (combined_cong["G20PRERTRU"] + combined_cong["G20PREDBID"]),3)


### Export to csv

In [9]:
if not os.path.exists("./national_20_pres_dem_rep_vote_share_22_leg_boundary"):
    os.mkdir("./national_20_pres_dem_rep_vote_share_22_leg_boundary")

combined_cong.to_csv('./national_20_pres_dem_rep_vote_share_22_leg_boundary/2022_cong_results.csv', index = False)
combined_SLDU.to_csv('./national_20_pres_dem_rep_vote_share_22_leg_boundary/2022_sldu_results.csv', index = False)
combined_SLDL.to_csv('./national_20_pres_dem_rep_vote_share_22_leg_boundary/2022_sldl_results.csv', index = False)

In [11]:
fields_dict = {'STATE':"State Abbreviation",
               'SLDL':"State Legislative District (Lower House) Name or Number",
               'SLDU':"State Legislative District (Upper House) Name or Number",
               'CONG':"Congressional District Name or Number",
               'SLDL-ID':"Unique State Legislative District (Lower House) ID",
               'SLDU-ID':"Unique State Legislative District (Upper House) ID",
               'CONG-ID':"Unique Congressional District ID",
               'P0010001':"Total Population from 2020 Census",
               'VAP_MOD':"Total Voting Age Population minus incarcerated adult population",
               'G20PRERTRU':"Total 2020 Trump Votes",
               'G20PREDBID':"Total 2020 Biden Votes",
               'DEM_pct':"Pct 2020 Biden Votes",
               'REP_pct':"Total 2020 Trump Votes"}

In [13]:
### Create README

title = "2020 Presidential Democratic & Republican Vote Share Aggregated to 2022 Legislative Districts"
retrieval_date = "01/17/23"
fields_dict = fields_dict
github_link = "https://github.com/nonpartisan-redistricting-datahub/Processing-Requests/tree/main/Dem_Rep_20_Pres_Results_22_Leg_Districts_01_13_23"
file_folder = "./national_20_pres_dem_rep_vote_share_22_leg_boundary/"

def full_readme_text(title, retrieval_date, fields_dict, github_link):

#First section of README
    readme_p1 = '''{title}\n
## RDH Date Retrieval
{retrieval_date}

## Sources
ADD SOURCE DATA HERE

## Fields:
'''.format(title = title, retrieval_date = retrieval_date)

#Second section of README
    fields_table = pd.DataFrame.from_dict(fields_dict.items())
    fields_table.columns = ["Field Name", "Description"]
    readme_p2 = fields_table.to_string(formatters={'Description':'{{:<{}s}}'.format(fields_table['Description'].str.len().max()).format, 'Field Name':'{{:<{}s}}'.format(fields_table['Field Name'].str.len().max()).format}, index=False, justify = "left")

#Third section of README
    readme_p3 = '''\n
## Processing Steps
Visit the RDH GitHub and the processing script for this code [here]({github_link})

Please direct questions related to processing this dataset to info@redistrictingdatahub.org.
'''.format(github_link=github_link)
    
    full_readme = str(readme_p1)+str(readme_p2)+str(readme_p3)
    return full_readme

with open(file_folder+"README.txt", 'w') as tf:
        tf.write(full_readme_text(title, retrieval_date, fields_dict, github_link))

